<a href="https://colab.research.google.com/github/trgreen/Ages/blob/master/AgES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Agricultural Ecosystems Services (AgES) model 
AgES is a component-based watershed model built with the of Java Connection Framework (JCF) and Object Model Systems (OMS) frameworks. Water is routed explicitly among Hydroligal Response Units (HRUs) and stream reaches with topology and routing generated by the Catchment areas delineation (Cadel) web tool for each project. 





## Application of the Agricultural Ecosystems Services (AgES) watershed model 
---
*Olaf David, Timothy Green, Holm Kipka, Nathan Ligthart*

## Objective
Illustrate how to run the AgES (pronounced "Ages") model-as-a-service:  
1.   Execute the AgES CSIP service hosted at Colorado State University through a Jupyter Notebook.
2.   Provide the required libraries using the Cloud Services Innovation Platform (CSIP).
3.   Explain the step-by-step model setup, data upload to the Jupyter Notebook, model execution, and service output download of generated data files.
4.   Results can be visualized, printed, or downloaded from the Jupyter notebook.
5.   Exercise some CSIP client API capabilities -- play with it!

A separate Jupyter notebook provides new model calibration methods.
https://colab.research.google.com/drive/1aHmgj0wD4PaND75rMi7Lm_zHv30t2E9F#scrollTo=4FzchYnv57df 


## References
* CSIP Platform: http://alm.engr.colostate.edu/cb/project/csip 
* AgES Model: https://alm.engr.colostate.edu/cb/project/ages 
** New! Install package for Windows machines (not for ages-as-a-service):
    https://alm.engr.colostate.edu/cb/dir/79132
* CSIP Python client: https://pypi.org/project/csip

## Data 
* Example data set: South Fork Iowa River (SFIR) watershed with predominantly corn-soybean crop rotations, tile drainage, and swine manure application. Observed data include streamflow and nitrate concentration at the outlet.
[sf-data.zip](https://alm.engr.colostate.edu/cb/doc/76469/content?raw=true)

## CSIP Serices
* OMS/AgES Model service


## Preprocessed spatial data (NOT demonstrated here) for model inputs
These services provide essential input files for AgES, not included in the present demonstration.  

###Catchment areas delineation (Cadel)
Full details are given at: https://alm.engr.colostate.edu/cb/project/cadel

Here is a streamlined overview of Cadel.
Project setup for AgES requires the user to delineate the watershed and internal simulation areas (HRUs = Hydrological Response Units) along with the routing scheme.  Cadel generates the following AgES input files (comma separated values):
* hrus.csv
* routing.csv
* reach.csv

Note that these are flat ASCII files, but Cadel also produces graphical files to visualize the delineation and topology via the interface.

The workflow for Cadel follows:

1. Upload files to Cadel

[](https://www.dropbox.com/s/w57da1cevnf85li/Cadel1.png?dl=1)

2.  Fill sinks in the DEM
[](https://www.dropbox.com/s/7jao7td6xguvsqi/Cadel2.png?dl=1)

3. Delineate watershed boundary
[](https://www.dropbox.com/s/i470b1kbps6u3u2/Cadel3.png?dl=1)

4. Sub-catchment Delineation (SD) based on stream network
[](https://www.dropbox.com/s/t9m3o91noxp4c6t/Cadel4SD.png?dl=1)

5. Generate raw HRUs using overlay disection
[](https://www.dropbox.com/s/yl609f9h7nsuwbm/Cadel5Overlay.png?dl=1)

6. Final HRUs (coarsened based on minimum HRU area)
[](https://www.dropbox.com/s/fmrke1y8lwmc1kg/Cadel6HRU.png?dl=1)

7. Topology with flow vectors
[](https://www.dropbox.com/s/y0hn42q3jvdytbj/Cadel7topo.png?dl=1)

8.  Generate AgES input files (HRUs, stream reaches, topology/routing) 
[](https://www.dropbox.com/s/bjibhne6loc6xoc/Cadel9output.png?dl=1)

Finally, Cadel generates AgES input .csv files (HRUs, stream reaches, topology/routing), and vector data can be saved to view watershed delineation images.

Example of 30 HRU catchment areas delineated for the SFIR example:

[](https://www.dropbox.com/s/fmrke1y8lwmc1kg/Cadel6HRU.png?dl=1)
![](https://www.dropbox.com/s/nw6fxe38uzxacp8/SFIR30HRUs1.png?dl=1)

###Landuse and Agricultural Management Practices web-Service (LAMPS)
LAMPS queries public crop and irrigation maps and land management default data for specific landuse and crop rotations (USA only) to build the following AgES input files:
LAMPS generates the following AgES input files (comma separated values):
* croprotation.csv
* management.csv
* tillage.csv

The crop rotation file (croprotation.csv) specifies a sequence of crop management rotations. Every crop management rotation sequence has a Rotation ID (RID) and a list of management IDs (MIDs) which are linked to the management file. The tillage file (tillage.csv) contains tillage operation data including tillage efficiency and depth for each operation. The management file (management.csv) combines crop-specific information about management operations and practices for specific dates. The management file connects the crop rotation, crop management rotation and tillage files.

AgES input files (Cadel and LAMPS output files) are included in the example project data:
[sf-data.zip](https://alm.engr.colostate.edu/cb/doc/76469/content?raw=true)



## Setup for AgES service calls



###  CSIP Library Installation


Install the CSIP Python client library from PyPI. Use the `pip3` command to install the latest.

In [1]:
!pip3 install csip --no-cache-dir --upgrade

  Created wheel for csip: filename=csip-1.0.6-cp36-none-any.whl size=9563 sha256=e0a09da409ac06585bc86eee78dead369a105bf5ea23560c51f1c85417b3e65a
  Stored in directory: /tmp/pip-ephem-wheel-cache-4kiu9ewb/wheels/d9/d5/5f/d84a8fbd5e7782eabb331c0c366606a6019cea10fa17bb6b02
Successfully built csip


In [2]:
!ls -al

total 16
drwxr-xr-x 1 root root 4096 Jun 10 16:28 .
drwxr-xr-x 1 root root 4096 Jun 11 22:40 ..
drwxr-xr-x 1 root root 4096 Jun 10 16:28 .config
drwxr-xr-x 1 root root 4096 Jun 10 16:28 sample_data


### Get Example Data

In the first step the data is  uploaded to the notebook.

Examples below use '3sf.zip' as a 3-HRU representation of the South Fork Iowa River watershed.

In [3]:
!wget -U "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:40.0) Gecko/20100101 Firefox/40.0" \
     --content-disposition "https://alm.engr.colostate.edu/cb/doc/76469/content?raw=true"

--2020-06-11 22:54:23--  https://alm.engr.colostate.edu/cb/doc/76469/content?raw=true
Resolving alm.engr.colostate.edu (alm.engr.colostate.edu)... 129.82.20.49
Connecting to alm.engr.colostate.edu (alm.engr.colostate.edu)|129.82.20.49|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 2246403 (2.1M) [application/zip]
Saving to: ‘sf-data.zip’

sf-data.zip         100%[===================>]   2.14M  2.10MB/s    in 1.0s    

2020-06-11 22:54:26 (2.10 MB/s) - ‘sf-data.zip’ saved [2246403/2246403]





---



Let's look at the content of `sf-data.zip`

In [4]:
!unzip  sf-data.zip

Archive:  sf-data.zip
   creating: data/
  inflating: data/regionalization.csv  
  inflating: data/management_tomerNEW19.csv  
  inflating: data/obs_data00_15.csv  
  inflating: data/obs_data02_15.csv  
  inflating: data/obs_data08_10.csv  
  inflating: data/reach.csv          
  inflating: data/crop.csv           
  inflating: data/croprotation.csv   
  inflating: data/fert.csv           
  inflating: data/till.csv           
  inflating: data/orun.csv           
  inflating: data/tmin.csv           
  inflating: data/tmax.csv           
  inflating: data/obs_data.csv       
  inflating: data/hrus_additional.csv  
  inflating: data/soils_hor.csv      
  inflating: data/hrus_rot.csv       
  inflating: data/precip.csv         
  inflating: data/obs_data08_12.csv  
  inflating: data/obs_data00_08.csv  
  inflating: data/hrus_rot_08_12.csv  
  inflating: data/solrad.csv         
  inflating: data/routing.csv        
  inflating: data/obs_data07_14.csv  
  inflating: data/wind.csv        

## CSIP Client for AgES 

### Library Import

Import the CSIP  Client library and instatiate a Client object.We call it `req` for request.

In [5]:
import csip
from csip.utils import Client
print(csip.__version__)

1.0.6


### Provide model inputs

Add inputs to the request object. 

In [6]:
req = Client()
req.add_data("loglevel", "SEVERE")
req.add_data("startTime", "2003-10-01")
req.add_data("endTime", "2005-10-30")
req.add_data("flowRouteTA", "1.2")

req.add_data("kge", ('data/obs_data02_15.csv/obs/orun[1]',
                     'output/csip_run/out/Outlet.csv/output/catchmentSimRunoff'))

print(req)

SERVICE: '' 
 url: None
 description: 
 parent: None 
 metainfo: {
             "service_url": null,
             "description": "",
             "name": ""
           }
 data:
  loglevel
      "SEVERE"
  startTime
      "2003-10-01"
  endTime
      "2005-10-30"
  flowRouteTA
      "1.2"
  kge
      [
         "data/obs_data02_15.csv/obs/orun[1]",
         "output/csip_run/out/Outlet.csv/output/catchmentSimRunoff"
       ]



### Execute the Model Service

Perform the model run by invoking the `HTTP/POST` service. 
The service uses:

*   The request parameter entries 
*   The CSIP URL endpoint
*   The files to attach as a string array.

The service will return on model completion.


In [7]:
resp = req.execute('http://csip.engr.colostate.edu:8087/csip-oms/m/ages/0.3.0', 
                files= ["sf-data.zip"])

print(resp)

SERVICE: '' 
 url: http://csip.engr.colostate.edu:8087/csip-oms/m/ages/0.3.0
 description: 
 parent: None 
 metainfo: {
             "service_url": "http://csip.engr.colostate.edu:8087/csip-oms/m/ages/0.3.0",
             "description": "",
             "name": "",
             "mode": "sync",
             "attachments": "request.json, data/regionalization.csv, data/management_tomerNEW19.csv, data/obs_data00_15.csv, data/obs_data02_15.csv, data/obs_data08_10.csv, data/reach.csv, data/crop.csv, data/croprotation.csv, data/fert.csv, data/till.csv, data/orun.csv, data/tmin.csv, data/tmax.csv, data/obs_data.csv, data/hrus_additional.csv, data/soils_hor.csv, data/hrus_rot.csv, data/precip.csv, data/obs_data08_12.csv, data/obs_data00_08.csv, data/hrus_rot_08_12.csv, data/solrad.csv, data/routing.csv, data/obs_data07_14.csv, data/wind.csv, data/obs_data08_14.csv, data/hgeo.csv, data/hrus.csv, data/hrus_rot_00_12.csv, data/soils_hor_kf_calib.csv, data/hru_override.csv, data/obs_data02_14.csv, 

### Download the output files

The service created multiple output that are listed in the response JSON. The `download_data_files()` method will fetch  and save them into the current directory.  

In [8]:
files = resp.get_data_files()
resp.download_data_files(files)

downloading to '/content':
  --> java-2119373119-stderr.txt
  --> java-2119373119-stdout.txt
  --> Outlet.csv


### Print the  Model's `stdout`

Use the `cat` command to print out the output. 

In [9]:
!cat java-*-stdout.txt

--> Executing /tmp/csip/bin/ages/simulation/ages.sim
--> Number of available processors: 32
--> Number of threads: 31
--> Reading HRU input file
--> Finished Reading HRUs [29]
--> Reading stream reach input file
--> Finished Reading Reaches [78]
--> Reading landuse input file
--> Finished reading landuses [12]
--> Reading hydrogeology input file
--> Finished reading hydrogeologies [27]
--> Reading soil input file ...
--> Reading fertilizer parameter database ...
--> Reading tillage parameter database ...
--> Reading crops parameter database ...
--> Reading management input file ...
--> Reading crop rotation input file ...
--> Linking crop rotation and management information ...
--> Initializing contexts ...
--> Finding watershed outlet: 
--> Found watershed outlet: -101
--> Creating programs ...
--> Process initialization completed ... 84
--> Preprocessing start ...
2003-10-01 [2568 ms]
2003-11-01 [558 ms]
2003-12-01 [299 ms]
2004-01-01 [176 ms]
2004-02-01 [153 ms]
2004-03-01 [107 ms]


## 4. Output Visualization
Preliminary example plots (in progress)


In [10]:
!pip3 install plotly

In [11]:
!head -10 Outlet.csv

@T, output
created_at,"Thu Jun 11 16:54:34 MDT 2020"
date_format,yyyy-MM-dd
@H,time,catchmentSimRunoff,catchmentSimRunoff_NO3_N,catchmentNO3_N_Load_kg_ha
Type,LocalDate,Double,Double,Double
Unit,-,m3/s,mg/L,kg/ha
,2003-10-01,0.00002,0,0
,2003-10-02,0.00014,0,0
,2003-10-03,0.00049,1.99579,0
,2003-10-04,0.00155,3.87929,0.00001


In [12]:
!head -20 data/obs_data02_15.csv

@T,obs,,,,,,,,,,,
descr,observed runoff at Drake,,,,,,,,,,,
unit,m3/s  //  mg/l  //  mg/l ,,,,,,,,,,,
,,,,,,,,,,,,
missing_val,-9999,,,,,,,,,,,
date_start,09.01.2001,,,,,,,,,,,
date_end,30.03.2015,,,,,,,,,,,
date_format,dd.MM.yyyy,,,,,,,,,,,
,,,,,,,,,,,,
@H,date,orun[1],no3_n[1],sedim[1],no3_n[2],no3_n[3],no3_n_load[4],no3_n_load_ntrax[4],sedim_load[2],tile[1],no3_n[4],no3_n_load[5]
name,-,SF450,SF450,SF450,SF450,SF450,SF450,SF450,SF450,SF450,SF450,SF450
ID,-,1,2,3,4,5,6,7,8,1,4,6
elevation,-,289,289,289,289,289,289,289,289,289,289,289
x,-,487469.4,487469.4,487469.4,487469.4,487469.4,487469.4,487469.4,487469.4,487469.4,487469.4,487469.4
y,-,4684771.7,4684771.7,4684771.7,4684771.7,4684771.7,4684771.7,4684771.7,4684771.7,4684771.7,4684771.7,4684771.7
,01.01.2002,1.557426563,17.31,-9999,-9999,-9999,-9999,-9999,-9999,1.00E+11,-9999,-9999
,02.01.2002,1.33089179,17.61,-9999,-9999,-9999,-9999,-9999,-9999,1.00E+11,-9999,-9999
,03.01.2002,1.472476023,17.9,-9999,-9999,-9999,-9999,-9999,-9999,1.0

In [13]:
!ls -l

total 2240
drwxrwxr-x 2 root root    4096 Feb 10 22:49 data
-rw-r--r-- 1 root root     217 Jun 11 22:54 java-2119373119-stderr.txt
-rw-r--r-- 1 root root    1403 Jun 11 22:54 java-2119373119-stdout.txt
-rw-r--r-- 1 root root   28081 Jun 11 22:54 Outlet.csv
drwxr-xr-x 1 root root    4096 Jun 10 16:28 sample_data
-rw-r--r-- 1 root root 2246403 Jun 11 22:54 sf-data.zip


In [14]:
import pandas as pd
import plotly.express as px
output = pd.read_csv("Outlet.csv", sep=',',  skiprows=3, header=[0,1,2]).dropna(axis=1)
x = output[output.columns[0]]
output


,time,catchmentSimRunoff,catchmentSimRunoff_NO3_N,catchmentNO3_N_Load_kg_ha
,LocalDate,Double,Double,Double
,-,m3/s,mg/L,kg/ha
0,2003-10-01,0.00002,0.00000,0.00000
1,2003-10-02,0.00014,0.00000,0.00000
2,2003-10-03,0.00049,1.99579,0.00000
3,2003-10-04,0.00155,3.87929,0.00001
4,2003-10-05,0.00508,5.73833,0.00004
...,...,...,...,...
756,2005-10-26,0.15362,6.41482,0.00146
757,2005-10-27,0.14482,6.46308,0.00138


In [15]:
y = output[output.columns[1]]
fig = px.line(x=x, y=y, title=y.name[0] + " [" + y.name[2] + "]")
fig.show()

In [16]:
import plotly.express as px
y = output[output.columns[2]]
fig = px.line(x=x, y=y, title=y.name[0] + " [" + y.name[2] + "]")
fig.show()

In [17]:
import plotly.express as px
y = output[output.columns[3]]
fig = px.line(x=x, y=y, title=y.name[0] + " [" + y.name[2] + "]")
fig.show()